In [ ]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gdown
from thefuzz import process
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
#Downloading data
file_id_1 = '1s0zEHlvgbIr4AKruko2yFQymK_Pp1QNi'
file_id_2 = '1sTnKn_XSgZ_8uzNo8INDtmz9WjIqziYG'
file_id_3 = '1WuCNd7d4ZUJ8OQPmGtvMk_2Fasr9v0v2'
url_1 = f"https://drive.google.com/uc?id={file_id_1}"
url_2 = f"https://drive.google.com/uc?id={file_id_2}"
url_3 = f"https://drive.google.com/uc?id={file_id_3}"
output_1 = 'data_dv.csv'
output_2 = 'data_ev.csv'
output_3 = 'data_ur.csv'
gdown.download(url_1, output_1, quiet=False)
gdown.download(url_2, output_2, quiet=False)
gdown.download(url_3, output_3, quiet=False)
df1 = pd.read_csv(output_1)
df2 = pd.read_csv(output_2)
df3 = pd.read_csv(output_3,sep = ';')

# Viewing data
df1.head()
df2.head()
df3.head()

#Cleaning data and changing name col
df1['disitrict_name'] = df1['disitrict_name'].astype(str)
df2['Disitrict'] = df2['Disitrict'].astype(str)
df2['district_clean'] = df2['Disitrict'].apply(lambda x:process.extractOne(x,df1['disitrict_name'])[0])
df3.drop(columns=['Kod','Unnamed: 3'], inplace=True)
df3.rename(columns={'Nazwa':'district_name','ogółem;2023;[%]':'unemployment_rate'},inplace=True)
df3['district_clean3'] = df3['district_name'].apply(lambda x: process.extractOne(x,df1['disitrict_name'])[0])

#Merging data into one DF
df_merged = df1.merge(df2,left_on='disitrict_name',right_on='district_clean',how='inner')
df_merged = df_merged.merge(df3,left_on='disitrict_name',right_on='district_clean3', how='inner')
df_merged.head()
df_merged.rename(columns = {'disitrict_name':'district_name'},inplace=True)
df_merged.drop(columns=['Disitrict','district_clean','district_clean3'], inplace=True)
df_merged.dropna(inplace=True)
df_merged.info()
float_list = ['salary','education_expenses']
for i in float_list:
    df_merged[i] = df_merged[i].astype(str)
    df_merged[i] = df_merged[i].apply(lambda x:x.replace(',',''))
    df_merged[i] = df_merged[i].astype(float)

df_merged['unemployment_rate'] = df_merged['unemployment_rate'].str.replace(',','.')
df_merged['unemployment_rate'] = df_merged['unemployment_rate'].astype(float)
df_merged.drop_duplicates(subset = ['district_name'], keep ='first', inplace = True)
df_merged.head()


In [28]:
#Linear regression OLS method without scaling
X = df_merged[['salary','education_expenses','unemployment_rate','divorces']]
y = df_merged['mean_score']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
model_OLS = LinearRegression()
model_OLS.fit(X_train, y_train)
R2 = model_OLS.score(X_test, y_test)
df_intercept = pd.DataFrame(model_OLS.intercept_,['Intercept'],columns=['Value'])
df_coef = pd.DataFrame(model_OLS.coef_,X.columns,columns=['Value'])
df_equation = pd.concat([df_intercept, df_coef],ignore_index=False)

# Validation of model
y_pred_OLS = model_OLS.predict(X_test)
mae_OLS = mean_absolute_error(y_test, y_pred)
mse_OLS = mean_squared_error(y_test, y_pred)
r2_OLS = r2_score(y_test, y_pred)
rmse_OLS = np.sqrt(mse_OLS)

# Creating data frane of results
df_results_OLS = pd.DataFrame([mae_OLS, mse_OLS, r2_OLS, rmse_OLS], index=['MAE','MSE','R2','RMSE'],columns=['Value'])
print(df_results_OLS)

          Value
MAE    5.855805
MSE   53.003729
R2     0.041051
RMSE   7.280366


In [ ]:
#plt.scatter(y=Y,x=df_merged['unemployment_rate'])